---
title: "Hazard-based distributional regression via ordinary differential equations: Breast cancer recurrence data"
author: '[F. Javier Rubio](https://sites.google.com/site/fjavierrubio67/)'
date: today
format:
  html:
    theme: flatly
    code-fold: true
    toc: true
    number-sections: true
execute:
  kernel: julia
  options:
    threads: 8  # Set the number of threads here
bibliography: references.bib
---



# Models

In this application, we model the recurrence (relapse) times of $n=2,939$ breast cancer patients from the `rotterdam` dataset in the R package `survival`. This is an observational dataset which includes information on recurrence times and censoring indicators (due to death or end of follow-up). We consider the following covariates: \texttt{age} (age at surgery, start of follow-up), `nodes` (number of positive lymph nodes), `size` (tumor size, dichotomised $0 =$ $\leq 50$, $1 =$ $>50$), and `trt` (chemotherapy, binary). The maximum follow-up duration is $19.3$ years, with $52\%$ of patients experiencing recurrence during the follow-up period. $580$ patients received chemotherapy, while $334$ patients also received hormonal treatment, with the majority ($306$) being on the non-chemotherapy group. 

Since some patients received treatment, it is reasonable to expect a reduction in the hazard associated with cancer recurrence. Consequently, we model this dataset using the hazard-response model, 

$$
  \begin{cases}
h'(t)  =  \lambda h(t) \left(1 - \dfrac{h(t)}{\kappa}\right) - \alpha q(t) h(t), & h(0) = h_0 \\
q'(t) =  \mu q(t) \left( 1- \dfrac{q(t)}{\kappa} \right) -\alpha q(t) h(t)  ,  &  q(0) = q_0 \\ 
H'(t)  =  h(t), & H(0) = 0,
\end{cases}\begin{cases}
h'(t)  =  \lambda h(t) \left(1 - \dfrac{h(t)}{\kappa}\right) - \alpha q(t) h(t), & h(0) = h_0 \\
q'(t) =  \mu q(t) \left( 1- \dfrac{q(t)}{\kappa} \right) -\alpha q(t) h(t)  ,  &  q(0) = q_0 \\ 
H'(t)  =  h(t), & H(0) = 0,
\end{cases}
$$
  
  incorporating covariates on the corresponding parameters as described in 

> Christen, J.A. and Rubio, F.J. (2025). Hazard-based distributional regression via ordinary differential equations. Submitted.

This selected regression model includes: $(\text{intercept}, \texttt{nodes}, \texttt{trt})$ for $\lambda$, $(\text{intercept}, \texttt{nodes})$ for $\kappa$, $(\text{intercept}, \texttt{nodes}, \texttt{trt})$ for $\alpha$, and $(\text{intercept}, \texttt{age}, \texttt{size}, \texttt{trt})$ for $\mu$.

################################################################################
################################################################################
# Julia code
################################################################################
################################################################################

## Required packages


In [ ]:
#=
****************************************************************************
Required packages
****************************************************************************
=#

using Plots
using DifferentialEquations
using LinearAlgebra
using CSV
using LSODA
using Optim
using Distributions
using Random
using AdaptiveMCMC
using Tables
using DelimitedFiles
using Statistics
using Survival
using DataFrames
using FreqTables
using Sundials
using ForwardDiff
using Turing
using StatsPlots
using StatsFuns
using JLD2

# Additional routines 
include("routines.jl")

## Data preparation


In [ ]:
#=
****************************************************************************
Data preparation
****************************************************************************
=#


#= Data =#
df_full = CSV.File("rotterdamFull.csv");
df0 = CSV.File("rotterdam.csv");


#= Data of interest =#
size_bin = collect(df_full.size)
sizes = zeros(size(size_bin)[1])

for i in 1:size(size_bin)[1]
    if size_bin[i] == ">50"
        sizes[i] = 1.0
    else
        sizes[i] = 0.0
    end
  end


df = DataFrame( time = collect(df_full.rtime)./365.25, 
                status = collect(df_full.recur),
                nodes = scale(collect(df_full.nodes)),
                agec = scale(collect(df_full.age)),
                sizes = sizes,
                trt = convert(Vector{Float64}, collect(df_full.chemo)))

# Removing inconsistent Data
indcon = zeros(size(df_full)[1])
for i in 1:size(df_full)[1]
    if ((df_full.rtime[i] < df_full.dtime[i]) && (df_full.recur[i] == 0) && (df_full.death[i] == 1))
        indcon[i] = 0
    else
        indcon[i] = 1
    end
end
indcon = collect(Bool, (indcon))

df = df[indcon,:]

# Sorting df by time
sorted_indices = sortperm(df[:, :time])

df = df[sorted_indices,:]

# Sample size
n = size(df)[1]

#= Vital status =#
status = collect(Bool, (df.status));

#= Survival times =#
times = df.time;

# Time grids
tspan0 = hcat(zeros(n), df.time);

tspan00 = vcat(0.0, df.time);
tmax = maximum(df.time)

# Initial conditions (h,q,H)
u0 = [1.0e-2, 1.0e-6, 0.0]

# Initial conditions (log h,log q,H)
lu0 = [log(1.0e-2), log(1.0e-6), 0.0]

#=
****************************************************************************
Data preparation for best model
****************************************************************************
=#

# Design matrix including variables of interest for model building
# age + sizes + nodes + trt
des_full = hcat( df.agec, df.sizes, df.nodes, df.trt)
  
# Design matrices for the regression models for each parameter 
#(intercept,age,sizes,nodes,trt)
# lambda: intercept + nodes + trt
des_l = hcat(ones(n), des_full[:,3:4]);
p_l = size(des_l)[2];
# kappa: intercept + nodes
des_k = hcat(ones(n), des_full[:,3]);
p_k = size(des_k)[2];
# alpha: intercept + nodes + trt
des_a = hcat(ones(n), des_full[:,3:4]);
p_a = size(des_a)[2];
# beta: intercept + age + sizes + trt
des_b = hcat(ones(n), des_full[:,[1,2,4]]);
p_b = size(des_b)[2];

# Intercept positions
indint = [1, p_l + 1, p_l + p_k + 1, p_l + p_k + p_a + 1]
indbeta = deleteat!(collect(1:(p_l+p_k+p_a+p_b)), indint)

# MLE vs. MCMC 


In [ ]:
# Reading the MLE

MLER = parse.(Float64, readlines("MLER.txt"))

# Reading the MCMC posterior sample

postsamp_mcmc = CSV.read("postsamp_mcmc.csv", Tables.matrix, header=true);


# Reading the normal approximation posterior sample

postsamp_app = readdlm("app_post_samples.txt");

# Comparison

hcat(MLER, vec(mean(postsamp_app,dims=1)), vec(mean(postsamp_mcmc,dims=1)))

In [ ]:
# Visualising results
plot_histograms_by_column(postsamp_mcmc)

################################################################################
################################################################################
# MLE Analysis
################################################################################
################################################################################


## Marginal survival probability of recurrence: Chemotherapy vs. No-Chemotherapy


In [ ]:
#=
-------------------------------------------------------------------------------------------------- 
# Marginal Survival: Chemotherapy vs. No-Chemotherapy
# Averaged over observed covariates, plug-in MLE.
-------------------------------------------------------------------------------------------------- 
=#

# Estimated parameters
paramsHR_MLE = hcat( exp.(des_l * MLER[1:p_l]),
                       exp.(des_k * MLER[(p_l+1):(p_l+p_k)]),
                       exp.(des_a * MLER[(p_l+p_k+1):(p_l+p_k+p_a)]),
                       exp.(des_b * MLER[(p_l+p_k+p_a+1):(p_l+p_k+p_a+p_b)]) )

    
#=
====================================
# No Chemotherapy group
===================================
=#

indNT = findall(x->x==0, df.trt)
nNT = length(indNT)



# Marginal survival function
function MNTSurv(t)
    OUT = zeros(Float64, nNT, 3)
    for i in 1:nNT
        sol = solve(ODEProblem(HRJ, u0, t, paramsHR_MLE[indNT[i], :]), Tsit5())
        OUT[i, :] = reduce(vcat, sol.u[end, :])
    end

    surp = mean(exp.(-OUT[:, 3]))

    return surp

end



#=
====================================
# Chemotherapy group
===================================
=#

indT = findall(x->x==1, df.trt)
nT = length(indT)



# Marginal survival function
function MTSurv(t)
    OUT = zeros(Float64, nT, 3)
    for i in 1:nT
        sol = solve(ODEProblem(HRJ, u0, t, paramsHR_MLE[indT[i], :]), Tsit5())
        OUT[i, :] = reduce(vcat, sol.u[end, :])
    end

    surp = mean(exp.(-OUT[:, 3]))

    return surp

end



# Kaplan-Meier estimator 
km_NT = fit(KaplanMeier, df.time[indNT], df.status[indNT])
km_T = fit(KaplanMeier, df.time[indT], df.status[indT])

ktimesNT = sort(unique(times[indNT]))
ktimesT = sort(unique(times[indT]))
ksurvival_probsNT = km_NT.survival
ksurvival_probsT = km_T.survival

# Comparison
plot(ktimesNT, ksurvival_probsNT,
    xlabel = "Time (years)", ylabel = "Population Survival", title = "",
  xlims = (0.0001,maximum(times)),   xticks = 0:2:maximum(times), label = "", 
  xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
  xguidefontsize=18, yguidefontsize=18, linewidth=3,
  linecolor = "gray", ylims = (0,1), linestyle=:solid)

plot!(ktimesT, ksurvival_probsT,
    xlabel = "Time (years)", ylabel = "Population Survival", title = "",
  xlims = (0.0001,maximum(times)),   xticks = 0:2:maximum(times), label = "", 
  xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
  xguidefontsize=18, yguidefontsize=18, linewidth=3,
  linecolor = "gray", ylims = (0,1), linestyle=:solid)


plot!(t -> MNTSurv(t),
    xlabel="Time (years)", ylabel="Survival", title="",
    xlims=(0, maximum(times)), xticks=0:2:20, label="",
    xtickfont=font(16, "Courier"), ytickfont=font(16, "Courier"),
    xguidefontsize=18, yguidefontsize=18, linewidth=3,
    linecolor="blue", legend = true, labels= "No Chemotherapy")

plot!(t -> MTSurv(t), xlims=(0, maximum(times)), 
    xguidefontsize=18, yguidefontsize=18, linewidth=3,
    linecolor = "red", linestyle=:solid, legend = true, labels = "Chemotherapy")

## Marginal hazards: Chemotherapy vs. No-Chemotherapy


In [ ]:
# Marginal hazard function
function NTMhaz(t)
OUT = zeros(Float64, nNT, 3)
for i in 1:nNT
sol = solve(ODEProblem(HRJ, u0, t, paramsHR_MLE[indNT[i], :]), Tsit5())
OUT[i, :] = reduce(vcat, sol.u[end, :])
end

mhaz = mean(exp.(-OUT[:, 3]) .* OUT[:, 1]) / mean(exp.(-OUT[:, 3]))

return mhaz

end



#=
====================================
# Chemotherapy group
===================================
=#
  

# Marginal hazard function
function TMhaz(t)
OUT = zeros(Float64, nT, 3)
for i in 1:nT
sol = solve(ODEProblem(HRJ, u0, t, paramsHR_MLE[indT[i], :]), Tsit5())
OUT[i, :] = reduce(vcat, sol.u[end, :])
end

mhaz = mean(exp.(-OUT[:, 3]) .* OUT[:, 1]) / mean(exp.(-OUT[:, 3]))

return mhaz

end

In [ ]:
plot(t -> NTMhaz(t),
      xlabel="Time (years)", ylabel="Hazard", title="",
      xlims=(0, maximum(times)), xticks=0:2:20, label="",
      xtickfont=font(16, "Courier"), ytickfont=font(16, "Courier"),
      xguidefontsize=18, yguidefontsize=18, linewidth=3,
      linecolor="blue", legend = true, labels= "No Chemotherapy")

plot!(t -> TMhaz(t), xlims=(0, maximum(times)), 
      xguidefontsize=18, yguidefontsize=18, linewidth=3,
      linecolor = "red", linestyle=:solid, legend = true, labels = "Chemotherapy")

## mean age + 0 size (<=50) + min nodes: Chemotherapy vs. No-Chemotherapy


In [ ]:
#--------------------------------------------------------------------------------------------------
# : Chemotherapy vs. No-Chemotherapy
#--------------------------------------------------------------------------------------------------

# No Chemotherapy group
valx_lNTE = [1.0, minimum(df.nodes), 0.0]
valx_kNTE = [1.0, minimum(df.nodes)]
valx_aNTE = [1.0, minimum(df.nodes), 0.0]
valx_bNTE = [1.0, 0.0, 0.0, 0.0]

lambdaNTE = exp.(valx_lNTE'MLER[1:p_l])
kappaNTE = exp.(valx_kNTE'MLER[(p_l+1):(p_l+p_k)])
alphaNTE = exp.(valx_aNTE'MLER[(p_l+p_k+1):(p_l+p_k+p_a)])
betaNTE = exp.(valx_bNTE'MLER[(p_l+p_k+p_a+1):(p_l+p_k+p_a+p_b)])
parNTE = [lambdaNTE[1], kappaNTE[1], alphaNTE[1], betaNTE[1]]

# Chemotherapy group
valx_lTE = [1.0, minimum(df.nodes), 1.0]
valx_kTE = [1.0, minimum(df.nodes)]
valx_aTE = [1.0, minimum(df.nodes), 1.0]
valx_bTE = [1.0, 0.0, 0.0, 1.0]

lambdaTE = exp.(valx_lTE'MLER[1:p_l])
kappaTE = exp.(valx_kTE'MLER[(p_l+1):(p_l+p_k)])
alphaTE = exp.(valx_aTE'MLER[(p_l+p_k+1):(p_l+p_k+p_a)])
betaTE = exp.(valx_bTE'MLER[(p_l+p_k+p_a+1):(p_l+p_k+p_a+p_b)])
parTE = [lambdaTE[1], kappaTE[1], alphaTE[1], betaTE[1]]

# ODE solutions
solNTE = solve(ODEProblem(HRJ, u0, 35.0, parNTE, Tsit5()),saveat=0.05);
solTE = solve(ODEProblem(HRJ, u0, 35.0, parTE, Tsit5()),saveat=0.05);

solNTE = solve(ODEProblem(HRJ, u0, 35.0, parNTE); alg_hints=[:stiff],saveat=0.05);
solTE = solve(ODEProblem(HRJ, u0, 35.0, parTE); alg_hints=[:stiff],saveat=0.05);

In [ ]:
# Comparison: hazard functions
plot(solNTE.t,hcat(solNTE.u...)[1,:],
xlabel = "Time (years)", ylabel = "Hazard", title = "",
xlims = (0.0,maximum(times)),   xticks = 0:5:35, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims=(0, 0.1), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(solTE.t,hcat(solTE.u...)[1,:], linewidth=3,
linecolor = "blue", linestyle=:solid,
legend = true, labels = "Chemotherapy")

In [ ]:
# Comparison: response functions
plot(solNTE.t,hcat(solNTE.u...)[2,:],
xlabel = "Time (years)", ylabel = "Response", title = "",
xlims = (0.0,maximum(times)),   xticks = 0:5:35, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims=(0, 3.0), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(solTE.t,hcat(solTE.u...)[2,:], linewidth=3,
linecolor = "blue", linestyle=:solid,
legend = true, labels = "Chemotherapy")

In [ ]:
# Comparison: survival functions
plot(solNTE.t,exp.(-hcat(solNTE.u...)[3,:]),
xlabel = "Time (years)", ylabel = "Survival", title = "",
xlims = (0.0,maximum(times)),   xticks = 0:5:35, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims = (0,1), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(solTE.t,exp.(-hcat(solTE.u...)[3,:]), linewidth=3,
linecolor = "blue", linestyle=:solid,
legend = true, labels = "Chemotherapy")

## older age (1.5) + 1 size + larger number of nodes (3.0): Chemotherapy vs. No-Chemotherapy


In [ ]:
#--------------------------------------------------------------------------------------------------
# : Chemotherapy vs. No-Chemotherapy
#--------------------------------------------------------------------------------------------------

# No Chemotherapy group
valx_lNTL = [1.0, 3.0, 0.0]
valx_kNTL = [1.0, 3.0]
valx_aNTL = [1.0, 3.0, 0.0]
valx_bNTL = [1.0, 1.5, 1.0, 0.0]


lambdaNTL = exp.(valx_lNTL'MLER[1:p_l])
kappaNTL = exp.(valx_kNTL'MLER[(p_l+1):(p_l+p_k)])
alphaNTL = exp.(valx_aNTL'MLER[(p_l+p_k+1):(p_l+p_k+p_a)])
betaNTL = exp.(valx_bNTL'MLER[(p_l+p_k+p_a+1):(p_l+p_k+p_a+p_b)])
parNTL = [lambdaNTL[1], kappaNTL[1], alphaNTL[1], betaNTL[1]]

# Chemotherapy group
valx_lTL = [1.0, 3.0, 1.0]
valx_kTL = [1.0, 3.0]
valx_aTL = [1.0, 3.0, 1.0]
valx_bTL = [1.0, 1.5, 1.0, 1.0]

lambdaTL = exp.(valx_lTL'MLER[1:p_l])
kappaTL = exp.(valx_kTL'MLER[(p_l+1):(p_l+p_k)])
alphaTL = exp.(valx_aTL'MLER[(p_l+p_k+1):(p_l+p_k+p_a)])
betaTL = exp.(valx_bTL'MLER[(p_l+p_k+p_a+1):(p_l+p_k+p_a+p_b)])
parTL = [lambdaTL[1], kappaTL[1], alphaTL[1], betaTL[1]]

# ODE solutions
solNTL = solve(ODEProblem(HRJ, u0, 35.0, parNTL, Tsit5()),saveat=0.05);
solTL = solve(ODEProblem(HRJ, u0, 35.0, parTL, Tsit5()),saveat=0.05);

In [ ]:
# Comparison: hazard functions
plot(solNTL.t,hcat(solNTL.u...)[1,:],
xlabel = "Time (years)", ylabel = "Hazard", title = "",
xlims = (0.0,maximum(times)),   xticks = 0:5:35, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims = (0,0.6), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(solTL.t,hcat(solTL.u...)[1,:], linewidth=3,
linecolor = "blue", linestyle=:solid,
legend = true, labels = "Chemotherapy")

In [ ]:
# Comparison: response functions
plot(solNTL.t,hcat(solNTL.u...)[2,:],
xlabel = "Time (years)", ylabel = "Response", title = "",
xlims = (0.0,maximum(times)),   xticks = 0:5:35, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims = (0,0.5), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(solTL.t,hcat(solTL.u...)[2,:], linewidth=3,
linecolor = "blue", linestyle=:solid,
legend = true, labels = "Chemotherapy")

In [ ]:
# Comparison: survival functions
plot(solNTL.t,exp.(-hcat(solNTL.u...)[3,:]),
xlabel = "Time (years)", ylabel = "Survival", title = "",
xlims = (0.0,maximum(times)),   xticks = 0:5:35, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims = (0,1), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(solTL.t,exp.(-hcat(solTL.u...)[3,:]), linewidth=3,
linecolor = "blue", linestyle=:solid,
legend = true, labels = "Chemotherapy")

################################################################################
################################################################################
# Bayesian analysis
################################################################################
################################################################################


## Marginal Posterior predictive survival: Chemotherapy vs. No-Chemotherapy


In [ ]:
#--------------------------------------------------------------------------------------------------
# Posterior Predictive survival: Chemotherapy vs. No-Chemotherapy
# Averaged over covariates and poserior samples
#--------------------------------------------------------------------------------------------------

# Using the first k posterior samples for predictive inference
k = 250;
indp = 1:k;
#postsampp = copy(postsamp_mcmc)[indp, :];
postsampp = copy(postsamp_app)[indp, :];


# All posterior parameters
lambdap = exp.(postsampp[:, 1:p_l] * des_l')';
kappap = exp.(postsampp[:, (p_l+1):(p_l+p_k)] * des_k')';
alphap = exp.(postsampp[:, (p_l+p_k+1):(p_l+p_k+p_a)] * des_a')';
betap = exp.(postsampp[:, (p_l+p_k+p_a+1):(p_l+p_k+p_a+p_b)] * des_b')';


# Group 1: : No Chemotherapy

function MNTPSurv(t)
    # Function to calculate the individual survival
    function indfuncNTPS(patID)

        # Numerical solution for the ODE
        paramsHR = hcat(lambdap[patID, :],
            kappap[patID, :],
            alphap[patID, :],
            betap[patID, :])

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, paramsHR[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        surp = mean(exp.(-OUT[:, 3]))

        return surp
    end

    out = mean(indfuncNTPS.(indNT))

    return out
end


# Group 2: : Chemotherapy

function MTPSurv(t)
    # Function to calculate the individual survival
    function indfuncTPS(patID)

        # Numerical solution for the ODE
        paramsHR = hcat(lambdap[patID, :],
            kappap[patID, :],
            alphap[patID, :],
            betap[patID, :])

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, paramsHR[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        surp = mean(exp.(-OUT[:, 3]))

        return surp
    end

    out = mean(indfuncTPS.(indT))

    return out
end

# Grid of values
tps = [0.01:0.5:20.5;]
ntps = length(tps)

NTP_tps =  zeros(ntps);
for i in 1:ntps
    NTP_tps[i] = MNTPSurv(tps[i])
end

TP_tps =  zeros(ntps);
for i in 1:ntps
    TP_tps[i] = MTPSurv(tps[i])
end

In [ ]:
# Comparison
plot(ktimesNT, ksurvival_probsNT,
     xlabel = "Time (years)", ylabel = "Predictive survival", title = "",
     xlims = (0.0001,maximum(times)),   xticks = 0:2:maximum(times), label = "", 
     xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
     xguidefontsize=18, yguidefontsize=18, linewidth=3,
     linecolor = "gray", ylims = (0,1), linestyle=:solid)

plot!(ktimesT, ksurvival_probsT,
      xlabel = "Time (years)", ylabel = "Predictive survival", title = "",
      xlims = (0.0001,maximum(times)),   xticks = 0:2:maximum(times), label = "", 
      xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
      xguidefontsize=18, yguidefontsize=18, linewidth=3,
      linecolor = "gray", ylims = (0,1), linestyle=:solid)


plot!(tps,NTP_tps,
      xlabel="Time (years)", ylabel="Predictive survival", title="",
      xlims=(0, maximum(times)), xticks=0:2:20, label="",
      xtickfont=font(16, "Courier"), ytickfont=font(16, "Courier"),
      xguidefontsize=18, yguidefontsize=18, linewidth=3,
      linecolor="blue", legend = true, labels= "No Chemotherapy")

plot!(tps,TP_tps, xlims=(0, maximum(times)), 
      xguidefontsize=18, yguidefontsize=18, linewidth=3,
      linecolor = "red", linestyle=:solid, legend = true, labels = "Chemotherapy")

## Marginal Posterior predictive hazard: Chemotherapy vs. No-Chemotherapy


In [ ]:
# Group 1: : No Chemotherapy

function NTPMHaz(t)
    # Function to calculate the individual survival
    function indfuncNTPS(patID)

        # Numerical solution for the ODE
        paramsHR = hcat(lambdap[patID, :],
            kappap[patID, :],
            alphap[patID, :],
            betap[patID, :])

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, paramsHR[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        surp = mean(exp.(-OUT[:, 3]))

        return surp
    end
    
        function indfuncNTPden(patID)

        # Numerical solution for the ODE
        paramsHR = hcat(lambdap[patID, :],
            kappap[patID, :],
            alphap[patID, :],
            betap[patID, :])

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, paramsHR[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        mden = mean(exp.(-OUT[:, 3]) .* OUT[:, 1])

        return mden
    end

    out = mean(indfuncNTPden.(indNT))/mean(indfuncNTPS.(indNT))

    return out
end


# Group 2: : Chemotherapy


function TPMHaz(t)
    # Function to calculate the individual survival
    function indfuncTPS(patID)

        # Numerical solution for the ODE
        paramsHR = hcat(lambdap[patID, :],
            kappap[patID, :],
            alphap[patID, :],
            betap[patID, :])

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, paramsHR[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        surp = mean(exp.(-OUT[:, 3]))

        return surp
    end
    
        function indfuncTPden(patID)

        # Numerical solution for the ODE
        paramsHR = hcat(lambdap[patID, :],
            kappap[patID, :],
            alphap[patID, :],
            betap[patID, :])

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, paramsHR[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        mden = mean(exp.(-OUT[:, 3]) .* OUT[:, 1])

        return mden
    end

    out = mean(indfuncTPden.(indT))/mean(indfuncTPS.(indT))

    return out
end

# Grid of values
x1 = range(0.01, 5.0, length=80) # More points in this range
x2 = range(5.0, 20.0, length=20) # Fewer points here
tph = vcat(x1, x2)
ntph = length(tph)

NTP_tph =  zeros(ntph);
for i in 1:ntph
    NTP_tph[i] = NTPMHaz(tph[i])
end

TP_tph =  zeros(ntph);
for i in 1:ntph
    TP_tph[i] = TPMHaz(tph[i])
end

In [ ]:
# Comparison

plot(tph,NTP_tph,
      xlabel="Time (years)", ylabel="Predictive hazard", title="",
      xlims=(0, maximum(times)), xticks=0:2:20, label="",
      xtickfont=font(16, "Courier"), ytickfont=font(16, "Courier"),
      xguidefontsize=18, yguidefontsize=18, linewidth=3,
      linecolor="blue", legend = true, labels= "No Chemotherapy")

plot!(tph,TP_tph, xlims=(0, maximum(times)), 
      xguidefontsize=18, yguidefontsize=18, linewidth=3,
      linecolor = "red", linestyle=:solid, legend = true, labels = "Chemotherapy")

## Marginal Posterior predictive response: Chemotherapy vs. No-Chemotherapy


In [ ]:
#--------------------------------------------------------------------------------------------------
# Posterior Predictive response: Chemotherapy vs. No-Chemotherapy
# Averaged over covariates and poserior samples
#--------------------------------------------------------------------------------------------------

# Group 1: : No Chemotherapy

function MNTPResp(t)
    # Function to calculate the individual survival
    function indfuncNTPR(patID)

        # Numerical solution for the ODE
        paramsHR = hcat(lambdap[patID, :],
            kappap[patID, :],
            alphap[patID, :],
            betap[patID, :])

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, paramsHR[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        resp = mean(OUT[:, 2])

        return resp
    end

    out = mean(indfuncNTPR.(indNT))

    return out
end


# Group 2: : Chemotherapy

function MTPResp(t)
    # Function to calculate the individual survival
    function indfuncTPR(patID)

        # Numerical solution for the ODE
        paramsHR = hcat(lambdap[patID, :],
            kappap[patID, :],
            alphap[patID, :],
            betap[patID, :])

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, paramsHR[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        resp = mean(OUT[:, 2])

        return resp
    end

    out = mean(indfuncTPR.(indT))

    return out
end

# Grid of values
tpr1 = range(0.01, stop=5, length=25)
tpr2 = range(5, stop=20, length=15)

# Concatenate, removing duplicate at 5
tpr = vcat(tpr1, tpr2[2:end])
ntpr = length(tpr)

NTP_tpr =  zeros(ntpr);
for i in 1:ntpr
    NTP_tpr[i] = MNTPResp(tpr[i])
end

TP_tpr =  zeros(ntpr);
for i in 1:ntpr
    TP_tpr[i] = MTPResp(tpr[i])
end


# Comparison

plot(tpr,NTP_tpr,
      xlabel="Time (years)", ylabel="Predictive response", title="",
      xlims=(0, maximum(times)), xticks=0:2:20, label="",
      xtickfont=font(16, "Courier"), ytickfont=font(16, "Courier"),
      xguidefontsize=18, yguidefontsize=18, linewidth=3,
      linecolor="blue", legend = true, labels= "No Chemotherapy")

plot!(tpr,TP_tpr, xlims=(0, maximum(times)),
      xguidefontsize=18, yguidefontsize=18, linewidth=3,
      linecolor = "red", linestyle=:solid, legend = true, labels = "Chemotherapy")

## Conditional analysis. mean age + 0 size (<=50) + min nodes: Chemotherapy vs. No-Chemotherapy

### Posterior predictive hazard. 


In [ ]:
#=
--------------------------------------------------------------------------------------------------
# Predictive hazard: Chemotherapy vs. No-Chemotherapy
# Averaged over posterior samples, conditional on covariate values
--------------------------------------------------------------------------------------------------
=#

# Posterior parameters : No Chemotherapy group
lambdapNTE = exp.(postsampp[:, 1:p_l] * valx_lNTE)
kappapNTE = exp.(postsampp[:, (p_l+1):(p_l+p_k)] * valx_kNTE)
alphapNTE = exp.(postsampp[:, (p_l+p_k+1):(p_l+p_k+p_a)] * valx_aNTE)
betapNTE = exp.(postsampp[:, (p_l+p_k+p_a+1):(p_l+p_k+p_a+p_b)] * valx_bNTE)
parpNTE = hcat(lambdapNTE, kappapNTE, alphapNTE, betapNTE)

# Posterior parameters : Chemotherapy group
lambdapTE = exp.(postsampp[:, 1:p_l] * valx_lTE)
kappapTE = exp.(postsampp[:, (p_l+1):(p_l+p_k)] * valx_kTE)
alphapTE = exp.(postsampp[:, (p_l+p_k+1):(p_l+p_k+p_a)] * valx_aTE)
betapTE = exp.(postsampp[:, (p_l+p_k+p_a+1):(p_l+p_k+p_a+p_b)] * valx_bTE)
parpTE = hcat(lambdapTE, kappapTE, alphapTE, betapTE)

# Predictive hazard: No Chemotherapy group
function predHNTE(t)
        # Numerical solution for the ODE

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, parpNTE[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        hpred = mean(exp.(-OUT[:, 3]) .* OUT[:, 1]) / mean(exp.(-OUT[:, 3]))

        return hpred
end


# Predictive hazard: Chemotherapy group
function predHTE(t)

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, parpTE[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        hpred = mean(exp.(-OUT[:, 3]) .* OUT[:, 1]) / mean(exp.(-OUT[:, 3]))

        return hpred
end



# Comparison
tvec = [0.01:0.1:20.5;]
ntvec = length(tvec)


phnte = zeros(ntvec);
for i in 1:ntvec
    phnte[i] = predHNTE(tvec[i])
end

phte = zeros(ntvec);
for i in 1:ntvec
    phte[i] = predHTE(tvec[i])
end


# Comparison: hazard functions
plot(tvec,phnte,
xlabel = "Time (years)", ylabel = "Hazard", title = "Early",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims = (0,0.1), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(tvec,phte,
xlabel = "Time (years)", ylabel = "Hazard", title = "",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "blue", ylims = (0,0.1), linestyle=:solid,
legend = true, labels = "Chemotherapy")

### Response functions


In [ ]:
# Predictive response: No Chemotherapy group
function predRNTE(t)
        # Numerical solution for the ODE

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, parpNTE[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        rpred = mean(OUT[:, 2]) 

        return rpred
end


# Predictive response: Chemotherapy group
function predRTE(t)

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, parpTE[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        rpred = mean(OUT[:, 2]) 

        return rpred
end



prnte = zeros(ntvec);
for i in 1:ntvec
    prnte[i] = predRNTE(tvec[i])
end

prte = zeros(ntvec);
for i in 1:ntvec
    prte[i] = predRTE(tvec[i])
end


# Comparison: response functions
plot(tvec,prnte,
xlabel = "Time (years)", ylabel = "Response", title = "Early",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims = (0,2.75), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(tvec,prte,
xlabel = "Time (years)", ylabel = "Response", title = "",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "blue", ylims = (0,2.75), linestyle=:solid,
legend = true, labels = "Chemotherapy")

### Predictive survival functions


In [ ]:
# Predictive survival: No Chemotherapy group
function predSNTE(t)
        # Numerical solution for the ODE

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, parpNTE[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        spred = mean(exp.(-OUT[:, 3])) 

        return spred
end


# Predictive survival: Chemotherapy group
function predSTE(t)

        OUT = zeros(Float64, k, 3)
        for i in 1:k
            sol = solve(ODEProblem(HRJ, u0, t, parpTE[i, :]), Tsit5())
            OUT[i, :] = reduce(vcat, sol.u[end, :])
        end

        spred = mean(exp.(-OUT[:, 3])) 

        return spred
end



psnte = zeros(ntvec);
for i in 1:ntvec
    psnte[i] = predSNTE(tvec[i])
end

pste = zeros(ntvec);
for i in 1:ntvec
    pste[i] = predSTE(tvec[i])
end


# Comparison: survival functions
plot(tvec,psnte,
xlabel = "Time (years)", ylabel = "Survival", title = "Early",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims = (0,1), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(tvec,pste,
xlabel = "Time (years)", ylabel = "Survival", title = "",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "blue", ylims = (0,1), linestyle=:solid,
legend = true, labels = "Chemotherapy")

## Conditional analysis. older age (1.5) + 1 size + larger number of nodes (3.0): Chemotherapy vs. No-Chemotherapy


### Predictive hazards


In [ ]:
#=
--------------------------------------------------------------------------------------------------
# Predictive hazard: Chemotherapy vs. No-Chemotherapy
--------------------------------------------------------------------------------------------------
=#

# Posterior parameters : No Chemotherapy group
lambdapNTL = exp.(postsampp[:,1:p_l] * valx_lNTL)
kappapNTL = exp.(postsampp[:,(p_l+1):(p_l+p_k)] * valx_kNTL)
alphapNTL = exp.(postsampp[:,(p_l+p_k+1):(p_l+p_k+p_a)] * valx_aNTL)
betapNTL = exp.(postsampp[:,(p_l+p_k+p_a+1):(p_l+p_k+p_a+p_b)] * valx_bNTL)
parpNTL = hcat(lambdapNTL, kappapNTL, alphapNTL, betapNTL)

# Posterior parameters : Chemotherapy group
lambdapTL = exp.(postsampp[:,1:p_l] * valx_lTL)
kappapTL = exp.(postsampp[:,(p_l+1):(p_l+p_k)] * valx_kTL)
alphapTL = exp.(postsampp[:,(p_l+p_k+1):(p_l+p_k+p_a)] * valx_aTL)
betapTL = exp.(postsampp[:,(p_l+p_k+p_a+1):(p_l+p_k+p_a+p_b)] * valx_bTL)
parpTL = hcat(lambdapTL, kappapTL, alphapTL, betapTL)

# Predictive hazard: No Chemotherapy group
function predHNTL(t)

    OUT = zeros(Float64, k, 3)
    for i in 1:k
        sol = solve(ODEProblem(HRJ, u0, t, parpNTL[i, :]), Tsit5())
        OUT[i, :] = reduce(vcat, sol.u[end, :])
    end

    hpred = mean(exp.(-OUT[:, 3]).*OUT[:,1] ) / mean(exp.(-OUT[:, 3]) )

    return hpred
    end


# Predictive hazard: Chemotherapy group
function predHTL(t)

    OUT = zeros(Float64, k, 3)
    for i in 1:k
        sol = solve(ODEProblem(HRJ, u0, t, parpTL[i, :]), Tsit5())
        OUT[i, :] = reduce(vcat, sol.u[end, :])
    end

    hpred = mean(exp.(-OUT[:, 3]).*OUT[:,1] ) / mean(exp.(-OUT[:, 3]) )

    return hpred
    end



# Comparison
phntl = zeros(ntvec)
for i in 1:ntvec
    phntl[i] = predHNTL(tvec[i])
end



phtl = zeros(ntvec)
for i in 1:ntvec
    phtl[i] = predHTL(tvec[i])
end


# Comparison: hazard functions
plot(tvec,phntl,
xlabel = "Time (years)", ylabel = "Hazard", title = "Late",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims = (0,0.6), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(tvec,phtl,
xlabel = "Time (years)", ylabel = "Hazard", title = "",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "blue", ylims = (0,0.6), linestyle=:solid,
legend = true, labels = "Chemotherapy")

### Predictive responses


In [ ]:
#=
--------------------------------------------------------------------------------------------------
# Predictive response: Chemotherapy vs. No-Chemotherapy
--------------------------------------------------------------------------------------------------
=#


# Predictive response: No Chemotherapy group
function predRNTL(t)

    OUT = zeros(Float64, k, 3)
    for i in 1:k
        sol = solve(ODEProblem(HRJ, u0, t, parpNTL[i, :]), Tsit5())
        OUT[i, :] = reduce(vcat, sol.u[end, :])
    end

        rpred = mean(OUT[:, 2]) 

        return rpred
    end


# Predictive response: Chemotherapy group
function predRTL(t)

    OUT = zeros(Float64, k, 3)
    for i in 1:k
        sol = solve(ODEProblem(HRJ, u0, t, parpTL[i, :]), Tsit5())
        OUT[i, :] = reduce(vcat, sol.u[end, :])
    end

        rpred = mean(OUT[:, 2]) 

        return rpred
    end



# Comparison
prntl = zeros(ntvec)
for i in 1:ntvec
    prntl[i] = predRNTL(tvec[i])
end



prtl = zeros(ntvec)
for i in 1:ntvec
    prtl[i] = predRTL(tvec[i])
end


# Comparison: hazard functions
plot(tvec,prntl,
xlabel = "Time (years)", ylabel = "Response", title = "Late",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims = (0,0.5), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(tvec,prtl,
xlabel = "Time (years)", ylabel = "Response", title = "",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "blue", ylims = (0,0.5), linestyle=:solid,
legend = true, labels = "Chemotherapy")

### Predictive survival


In [ ]:
#=
--------------------------------------------------------------------------------------------------
# Predictive survival: Chemotherapy vs. No-Chemotherapy
--------------------------------------------------------------------------------------------------
=#


# Predictive survival: No Chemotherapy group
function predSNTL(t)

    OUT = zeros(Float64, k, 3)
    for i in 1:k
        sol = solve(ODEProblem(HRJ, u0, t, parpNTL[i, :]), Tsit5())
        OUT[i, :] = reduce(vcat, sol.u[end, :])
    end
        spred = mean(exp.(-OUT[:, 3])) 

        return spred
    end


# Predictive survival: Chemotherapy group
function predSTL(t)

    OUT = zeros(Float64, k, 3)
    for i in 1:k
        sol = solve(ODEProblem(HRJ, u0, t, parpTL[i, :]), Tsit5())
        OUT[i, :] = reduce(vcat, sol.u[end, :])
    end

        spred = mean(exp.(-OUT[:, 3])) 

        return spred
    end



# Comparison
psntl = zeros(ntvec)
for i in 1:ntvec
    psntl[i] = predSNTL(tvec[i])
end



pstl = zeros(ntvec)
for i in 1:ntvec
    pstl[i] = predSTL(tvec[i])
end


# Comparison: hazard functions
plot(tvec,psntl,
xlabel = "Time (years)", ylabel = "Survival", title = "Late",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "red", ylims = (0,1), linestyle=:solid,
legend = true, labels = "No Chemotherapy")

plot!(tvec,pstl,
xlabel = "Time (years)", ylabel = "Survival", title = "",
xlims = (0.0,20),   xticks = 0:5:20, label = "",
xtickfont = font(16, "Courier"),  ytickfont = font(16, "Courier"),
xguidefontsize=18, yguidefontsize=18, linewidth=3,
linecolor = "blue", ylims = (0,1), linestyle=:solid,
legend = true, labels = "Chemotherapy")